In [1]:
from langchain_community.llms import VLLM
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium import webdriver

from bs4 import BeautifulSoup
import pandas as pd

In [2]:
llm = VLLM(
    model="cpm-ai/Ocelot-Ko-self-instruction-10.8B-v1.0",
    tensor_parallel_size=8,
    temperature=0,
    top_k=50,
    top_p=0.95,
    max_model_len=1024,
    vllm_kwargs={'gpu_memory_utilization': 0.1}
)


/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-06-13 09:09:02,712	INFO worker.py:1749 -- Started a local Ray instance.


INFO 06-13 09:09:03 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='cpm-ai/Ocelot-Ko-self-instruction-10.8B-v1.0', speculative_config=None, tokenizer='cpm-ai/Ocelot-Ko-self-instruction-10.8B-v1.0', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=8, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=cpm-ai/Ocelot-Ko-self-instruction-10.8B-v1.0)
INFO 06-13 09:09:29 utils.py:618] Found nccl from library libnccl.so.2
INFO 06-13 09:09:29 pynccl.py:65] vLLM is using nccl==2.20.5
(RayWorkerWrapper pid=2718706) INFO 06-13 09:09:29 utils.py:618] Found nccl from library libnccl.so.2
(RayWorkerWra

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3", model_kwargs={"device": "cuda"})

/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [1]:
def generate_talent_description(company_name, retriever, llm):
    # 질의 생성
    query = f""" 
    "회사의 모든 인재상을 입력하세요."
    "인재상 설명 이외의 다른 단어는 출력하지 마세요."
    "질문에 최선을 다해 답변하세요. "
    "검색 가능한 모든 도구를 자유롭게 사용하세요 "
    "필요한 경우에만 관련 정보 제공을 하세요"
    "답을 모르면 모른다고 말하세요. 답을 지어내려고 하지 마세요."
    "반드시 한국어로 답변하세요"
    "다음과 같은 양식으로 답변하시오"
    "[인재상]"
    "1."
    "2."
    "n."
    """
    # 답변 생성
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
    )

    result = qa_chain({"query": query})
    talents = result["result"]

    return talents

In [2]:
# 검색 관련 함수
def extract_company_info(company_name):
    # 크롬 옵션 설정
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # WebDriver 설정
    driver = webdriver.Chrome(options=chrome_options)

    try:
        # 변수 설정
        base_url = "https://www.jobkorea.co.kr/starter/companyreport"

        # URL 구성 및 접속
        url = f"{base_url}?schTxt={company_name}"
        driver.get(url)

        # 원하는 요소가 로드될 때까지 기다리고 요소 찾기
        element = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, f'//dt/a/strong[contains(text(), "기업심층분석 1. {company_name}, 채용분석 및 기업정보")]'))
        )
    
        href = element.find_element(By.XPATH, '..').get_attribute('href')
        print(f"Found detailed analysis link: {href}")

        # 링크로 이동
        driver.get(href)

        # 페이지 소스 가져오기
        page_source = driver.page_source

        # BeautifulSoup으로 페이지 소스 파싱
        soup = BeautifulSoup(page_source, 'html.parser')

        # 'viewWrap' 클래스를 가진 모든 요소 찾기
        view_wrap_elements = soup.find_all(class_="viewWrap")

        # 각 요소에서 텍스트 추출
        extracted_text = ""
        for element in view_wrap_elements:
            text = element.get_text(strip=True)
            extracted_text += text + "\n"

        return extracted_text, href

    except Exception as e:
        return 'fail'

    finally:
        driver.quit()

In [3]:
from fastapi import FastAPI, Request
from fastapi.responses import StreamingResponse
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn
import nest_asyncio

In [4]:
csv_file_path = "./company_talents.csv"

In [5]:
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)



class InputText(BaseModel):
    item: str



@app.post('/get_talent')
async def edit_orion(input_text: InputText):
    company_name = input_text.item
    print('회사명 확인', company_name)
    df = pd.read_csv(csv_file_path)
    company_talent = ''
    if company_name in df["company_name"].values:
        talents = df[df["company_name"] == company_name]["talents"].values[0]
        company_talent = talents
        print("Talents:", talents)
        return {"talent": company_talent, "iscsv": 'true'}
    else:
        company_info = extract_company_info(company_name)
        if company_info == 'fail':
            return '회사를 찾을 수 없습니다'
        else :
            vector_store = FAISS.from_texts(texts=[company_info[0]], embedding=embeddings)
            retriever = vector_store.as_retriever(search_kwargs={"k": 1})
            talents = generate_talent_description(company_name, retriever, llm)
            company_talent = talents
            company_url = company_info[1]


            new_data = {"company_name": company_name, "talents": talents}
            df = pd.concat([df, pd.DataFrame([new_data])], ignore_index=True)
            
            # 결과를 CSV 파일에 저장
            df.to_csv(csv_file_path, index=False)
            print("인재상이 생성되어 CSV 파일에 추가되었습니다.")            
        
    return {"talent": company_talent, "url": company_url}




In [6]:
nest_asyncio.apply()

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8002, ssl_keyfile="/etc/letsencrypt/live/model.cpm-ai.site/privkey.pem", ssl_certfile="/etc/letsencrypt/live/model.cpm-ai.site/fullchain.pem")

INFO:     Started server process [2826948]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on https://0.0.0.0:8002 (Press CTRL+C to quit)


INFO:     165.154.32.235:47798 - "GET / HTTP/1.1" 404 Not Found
INFO:     165.154.32.235:47810 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     165.154.32.235:47814 - "GET /robots.txt HTTP/1.1" 404 Not Found
INFO:     165.154.32.235:47820 - "GET /sitemap.xml HTTP/1.1" 404 Not Found


INFO:     Shutting down


RuntimeError: Event loop stopped before Future completed.

In [52]:
company_name = '삼성전자'

In [53]:
# 크롬 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# WebDriver 설정
driver = webdriver.Chrome(options=chrome_options)

try:
    # 변수 설정
    base_url = "https://www.jobkorea.co.kr/starter/companyreport"

    # URL 구성 및 접속
    url = f"{base_url}?schTxt={'삼성전자'}"
    driver.get(url)

    # 원하는 요소가 로드될 때까지 기다리고 요소 찾기
    element = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, f'//dt/a/strong[contains(text(), "기업심층분석 1. {company_name}, 채용분석 및 기업정보")]'))
    )

    href = element.find_element(By.XPATH, '..').get_attribute('href')
    print(f"Found detailed analysis link: {href}")

    # 링크로 이동
    driver.get(href)

    # 페이지 소스 가져오기
    page_source = driver.page_source

    # BeautifulSoup으로 페이지 소스 파싱
    soup = BeautifulSoup(page_source, 'html.parser')

    # 'viewWrap' 클래스를 가진 모든 요소 찾기
    view_wrap_elements = soup.find_all(class_="viewWrap")

    # 각 요소에서 텍스트 추출
    extracted_text = ""
    for element in view_wrap_elements:
        text = element.get_text(strip=True)
        extracted_text += text + "\n"

    print(extracted_text)

finally:
    driver.quit()

Found detailed analysis link: https://www.jobkorea.co.kr/starter/companyreport/view?Inside_No=10931&schCtgr=101012&schGrpCtgr=101&schTxt=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&Page=1
삼성전자는 부문별로 나눠 신입 사원 및 경력 사원을 채용하고 있다. 신입 사원 모집은 상반기에는 3월, 하반기에는 9월에 지원서 접수가 시작된다. 특정 분야 인력이 필요할 경우, 삼성전자 홈페이지 및 주요 취업 사이트를 통해 채용 공고를 게재하고 있다. 또한 수시 모집도 진행하고 있으니 경력 지원서는 언제든지 등록하면 된다. 모든 채용은 삼성 채용 홈페이지를 통해 접수가 가능하다. 3급 신입 공채 지원은 졸업 예정자 또는 기 졸업자를 대상으로 한다. 병역필 또는 면제자로 해외여행에 결격 사유가 없어야 하고, 어학 자격(OPIc 또는 TOEIC Speaking)을 보유하고 있어야 한다.삼성전자, 기업 개요삼성전자는 본사를 거점으로 한국과 DX 부문 산하 해외 9개 지역총괄, DS 부문 산하 해외 5개 지역총괄의 생산/판매법인 및 SDC, Harman 산하 종속기업 등 232개의 종속기업으로 구성된 글로벌 전자 기업이다. 2021년 12월 조직 개편을 통해 CE 부문과 IM 부문을 DX 부문으로 통합했으며, 내부 조직 체계에 맞춰 SDC를 DS 부문과 별도로 구분했다. 제품의 특성에 따라 DX(Device eXperience), DS(Device Solutions) 2개의 부문과 패널 사업을 영위하는 SDC(삼성디스플레이㈜ 및 그 종속기업), 전장부품 사업 등을 영위하는 Harman(Harman International Industries, Inc. 및 그 종속기업)으로 나눠 경영을 하고 있다.#기업 정보_더보기삼성전자, 인재상열정(Passion)끊임없는 열정으로 미래에 도전하는 인재창의혁신(Creativity)창의와 혁신으로 세상을 변화시키는 

In [35]:
texts = ['박진수는 35살이다', '문승주는 18살이다', '최수환은 19살이다','최수환']

In [80]:
vector_store = FAISS.from_texts(texts=texts, embedding=embeddings)

In [81]:
retriever = vector_store.as_retriever(search_kwargs={"k": 1})

In [82]:
relevant_docs = retriever.get_relevant_documents(query='인재상')

In [83]:
relevant_docs

[Document(page_content='혁신으로 세상을 변화시키는 인재인간미/도덕성(Integrity)정직과 바른 행동으로 역할과 책임을 다하는 인재1. 서류 전형 패스하기지원서에 기본 인적 사항, 학업 이수 내용, 경험/자격, 에세이 등을 작성한 후 삼성 채용 홈페이지를 통해 접수하면 된다. 직무적합성평가는 지원 자격을 검토하는 서류 평가라고 볼 수 있다. 서류 전형에서는 기본적인 자격 요건과 이력서 검토를 통해 1차적인 역량 검증을 진행한다. 직무와 관련해 필요한 역량을 얼마나 준비했는지, 관심을 가지고 성실히 준비했는지를 평가한다. 인사 담당자들이 자기소개서에서 중점적으로 보는 것은 경험과 포부다. 지원하는 직무, 전공, 사업, 회사를 선택한 자신만의 이야기, 회사와 지원자가 동반 성장할 수 있는 요소, 혹은 어떻게 성장할 것인지 등에 대해 작성한다. 자신이 지원한 직군과 전공 관련성을 잘 녹여내는 것이 중요하다. 제출한 지원서를 바탕으로 직무 관련 사항을 다면평가하고, 합격자에 한해 직무적성검사 응시가')]

Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /home/ubuntu/.local/lib/python3.10/site-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2713139/517268756.py", line 4, in <module>
    uvicorn.run(app, host="0.0.0.0", port=8002, ssl_keyfile="/etc/letsencrypt/live/model.cpm-ai.site/privkey.pem", ssl_certfile="/etc/letsencrypt/live/model.cpm-ai.site/fullchain.pem")
  File "/home/ubuntu/.local/lib/python3.10/site-packages/uvicorn/main.py", line 577, in run
    server.run()
  File "/home/ubuntu/.local/lib/python3.10/site-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/home/ubuntu/.local/lib/python3.10/site-packages/nest_asyncio.py", line

In [54]:
from langchain_text_splitters import CharacterTextSplitter

In [55]:
text_splitter = CharacterTextSplitter(
    separator = '',
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
)


In [57]:
texts = text_splitter.split_text(extracted_text)

In [75]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [76]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
)

In [77]:
texts = text_splitter.split_text(extracted_text)

In [84]:
texts = ['박진수는 35살이다']
vector_store = FAISS.from_texts(texts=texts, embedding=embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# 질의 생성
query = f""" 
"회사의 모든 인재상을 입력하세요."
"인재상 설명 이외의 다른 단어는 출력하지 마세요."
"질문에 최선을 다해 답변하세요. "
"검색 가능한 모든 도구를 자유롭게 사용하세요 "
"필요한 경우에만 관련 정보 제공을 하세요"
"답을 모르면 모른다고 말하세요. 답을 지어내려고 하지 마세요."
"반드시 한국어로 답변하세요"
"다음과 같은 양식으로 답변하시오"
"[인재상]"
"1."
"2."
"n."
"""
# 답변 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

result = qa_chain({"query": query})


/home/ubuntu/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Processed prompts: 100%|██████████████████████████████████| 1/1 [00:10<00:00, 10.88s/it, Generation Speed: 10.85 toks/s]


In [87]:
qa_chain

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=VLLM(model='cpm-ai/Ocelot-Ko-self-instruction-10.8B-v1.0', tensor_parallel_size=8, temperature=0.0, top_p=0.95, top_k=50, vllm_kwargs={'gpu_memory_utilization': 0.1}, client=<vllm.entrypoints.llm.LLM object at 0x7fbc7c746410>)), document_variable_name='context'), return_source_documents=True, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f86687f6320>, search_kwargs={'k': 3}))

In [ ]:
talents = result["result"]


In [86]:
talents

' [인재상]\n1. 혁신적인 사고와 문제 해결 능력\n2. 팀워크와 협업 능력\n3. 리더십과 책임감\n4. 글로벌 마인드셋과 다문화 이해도\n5. 지속적인 학습과 자기 개발 의지\n6. 윤리적 가치와 도덕적 기준 준수\n7. 고객 중심의 사고와 서비스 정신\n8. 커뮤니케이션과 대인 관계 능력\n9. 도전 정신과 자기 주도적 태도\n10. 변화에 대한 적응력과 유연성\n\nConfidence: 90%'

In [ ]:
talents = generate_talent_description(company_name, retriever, llm)
company_talent = talents
company_url = company_info[1]

In [89]:
prompt = """마지막 질문에 답변하기 위해 다음 context를 사용하세요.
답을 모르겠으면 모르겠다고 말하세요. 답변을 지어낼려고 노력하지 마세요.

context
[
삼성전자는 부문별로 나눠 신입 사원 및 경력 사원을 채용하고 있다. 신입 사원 모집은 상반기에는 3월, 하반기에는 9월에 지원서 접수가 시작된다. 특정 분야 인력이 필요할 경우, 삼성전자 홈페이지 및 주요 취업 사이트를 통해 채용 공고를 게재하고 있다. 또한 수시 모집도 진행하고 있으니 경력 지원서는 언제든지 등록하면 된다. 모든 채용은 삼성 채용 홈페이지를 통해 접수가 가능하다. 3급 신입 공채 지원은 졸업 예정자 또는 기 졸업자를 대상으로 한다. 병역필 또는 면제자로 해외여행에 결격 사유가 없어야 하고, 어학 자격(OPIc 또는 TOEIC Speaking)을 보유하고 있어야 한다.삼성전자, 기업 개요삼성전자는 본사를 거점으로 한국과 DX 부문 산하 해외 9개 지역총괄, DS 부문 산하 해외 5개 지역총괄의 생산/  및 SDC, Harman 산하 종속기업 등 232개의 종속기업으로 구성된 글로벌 전자 기업이다. 2021년 12월 조직 개편을 통해 CE 부문과 IM 부문을 DX 부문으로 통합했으며, 내부 조직 체계에 맞춰 SDC를 DS 부문과 별도로 구분했다. 제품의 특성에 따라 DX(Device eXperience), DS(Device Solutions) 2개의 부문과 패널 사업을 영위하는 SDC(삼성디스플레이㈜ 및 그 종속기업), 전장부품 사업 등을 영위하는 Harman(Harman International Industries, Inc. 및 그 종속기업)으로 나눠 경영을 하고 있다.#기업 정보_더보기삼성전자, 인재상열정(Passion)끊임없는 열정으로 미래에 도전하는 인재창의혁신(Creativity)창의와 혁신으로 세상을 변화시키는 인재인간미/도덕성(Integrity)정직과 바른 행동으로 역할과 책임을 다하는 인재1. 서류 전형 패스하기지원서에 기본 인적 사항, 학업 이수 내용, 경험/자격, 에세이 등을 작성한 후 삼성 채용 홈페이지를 통해 접수하면 된다. 직무적합성평가는 지원 자격을 검토하는 서류 평가라고 볼 수 있다. 서류 전형에서는 기본적인 자격 요건과 이력서 검토를 통해 1차적인 역량 검증을 진행한다. 직무와 관련해 필요한 역량을 얼마나 준비했는지, 관심을 가지고 성실히 준비했는지를 평가한다. 인사 담당자들이 자기소개서에서 중점적으로 보는 것은 경험과 포부다. 지원하는 직무, 전공, 사업, 회사를 선택한 자신만의 이야기, 회사와 지원자가 동반 성장할 수 있는 요소, 혹은 어떻게 성장할 것인지 등에 대해 작성한다. 자신이 지원한 직군과 전공 관련성을 잘 녹여내는 것이 중요하다. 제출한 지원서를 바탕으로 직무 관련 사항을 다면평가하고, 합격자에 한해 직무적성검사 응시가 가능하다.삼성전자가 영위하고 있는 사업 분야는 기술 혁신의 속도가 상당히 빠르므로 장기적 관점에서 어떤 기술이 시대를 관통할지에 대해 감히 예측하기는 어렵다. 그럼에도 불구하고 시장, 소비자, 기술 동향 등의 전반 사항들을 참고해서 시간이 지났을 때 나는 삼성에서 어떤 것을 하고 있을 것이며 그 안에서 어떤 성과로써 회사에 기여하고 있을 것인지에 대한 구체적 비전을 제시해야 한다.전공별로 주의할 점으로는 이공계 전공자는 공학 인증제 수료 여부에 가산점이 있고, 전공의 과목 수, 난이도, 평점 등을 종합적으로 평가한다. 특히 연구개발, 기술, S/W 직군은 전공 이수 과목의 수와 난이도, 취득 성적 등 전공 능력을 종합적으로 평가해 전공을 충실히 이수한 지원자를 우대한다. 그 외 직무와 무관한 스펙은 일체 반영되지 않는다. 연구개발, 기술, S/W 직군은 수학, 물리 등의 기초 과목을 비롯해 전공 과목을 심화 과정까지 깊이 있게 공부하면 도움이 된다. 반면 문과 전공자는 자기소개서 또는 직무와 관련된 경험과 함께 직무 역량을 주로 평가한다. 영업마케팅, 경영지원 직군은 직무와 관련된 과목을 수강하고, 다양한 활동 경험을 쌓는 것이 도움이 된다.자기소개서에는 인사 담당자들이 지원자에 대해 관심을 가지고 궁금해할 만한 내용이 들어가면 좋다. 눈에 띄는 제목을 작성하거나 삼성전자에 꾸준한 관심을 가졌던 것을 증명해 줄 수 있는 사례들이 들어간다면 좋은 자기소개서가 될 것이다. 기사처럼 가독성 있게 쓰는 것이 좋고, 각 항목의 맥락이 이어지게 작성하는 것도 추천한다. 자기소개서는 면접에서 활용되므로, 반드시 본인의 이야기를 바탕으로 작성하고, 제출하기 전에 지원서 내용을 저장해 두고 면접 전에 반드시 다시 읽어보도록 한다.#합격 자소서_더 보기2. 필기 전형 패스하기삼성 직무적성검사(GSAT)는 단편적인 지식보다는 응시자의 문제 해결 능력, 논리적 사고 능력 등과 같이 주어진 상황을 유연하게 대처하고 해결할 수 있는 종합적인 능력을 평가하는 검사다. 연구개발, 기술/설비, 영업마케팅, 경영지원 지원자를 대상으로 수리력과 추리력을 평가한다. 2022년 시험에는 수리 20문항, 추리 30문항 등 총 50문항이 출제됐다. 2020년부터 온라인으로 진행하고 있어 독립된 응시 장소와 PC, 스마트폰이 필요하다. 원활한 검사 진행을 위해서 사전 예비 소집을 진행해 지원자의 응시 환경을 점검하고 있으며, 예비 소집에 참석하지 않은 응시자는 검사 당일 응시가 제한될 수 있다.GSAT 대비를 위해 시중에 나와 있는 문제집을 반복해 풀어보는 것을 권장한다. 난이도보다 제한 시간 내 빠르고 정확하게 푸는 게 관건인 시험인 만큼, 시간을 재어가며 푸는 연습을 하는 것이 좋다. 난이도는 매 채용마다 상이하다. 문항 수가 많으므로 모든 문제를 40~50초 이내에 풀어야 하는 만큼 시간 배분에 유의해야 한다. 시간이 부족하다고 해서 오답을 찍고 넘어가는 것은 좋은 방법이 아니다. 아는 문제만 정확하게 답하도록 한다.소프트웨어(SW) 직군 지원자를 대상으로 진행하는 SW 역량 테스트는 SW 문제 해결 역량을 측정하기 위해 도입됐다. PC를 통해 주어진 문제를 직접 코딩해 해결하는 실기 방식의 테스트다. 사용 가능한 프로그램 언어는 C, C++, Java, Python이며, 총 2개 문제를 해결하는 데 주어지는 시간은 240분이다. ‘SW Expert 아카데미’를 통해서 학습 및 모의 문제 풀이를 진행할 수 있다.디자인 포트폴리오 심사는 디자인 직군 지원자의 디자인 역량을 평가하기 위한 전형이다. 직무적합성평가 결과 발표 이후 직무별로 서식을 지정해 주고 있으며, 지원자는 주어진 요구 사항에 맞춰 개인/팀 작품을 포트폴리오로 제작해 제출하면 된다.#인적성 후기_더 보기3. 면접 전형 패스하기면접은 임원 면접, 직무 역량 면접, 창의성 면접으로 3차에 걸쳐 진행한다. 면접 방식은 3차 모두 면접자 1인과 면접위원 여러 명으로 구성된 개인별 면접 방식으로 이뤄진다. 자유로운 질의응답 형식으로 진행하는 임원 면접은 개인의 품성, 조직 적합성 등을 눈여겨 평가한다. 자기소개서를 바탕으로 면접이 이뤄지니 자신이 작성한 내용을 충분히 숙지하도록 한다. 질문의 의도를 잘 생각하고 사고의 유연성을 드러내는 답변을 하는 것이 좋다.직무 역량 면접은 전공 역량, 직무 동기 등을 평가하며 전공별 문제 풀이를 한 후 프레젠테이션 및 질의응답하는 방식으로 진행된다. 현업 실무자들이 면접관으로 참여해 업무 수행 능력을 평가하기 때문에, 주어지는 질문에 직무와 관련된 경험을 바탕으로 응답하는 것이 좋다.창의성 면접은 독창적인 아이디어와 논리 전개 과정에 중점을 두어 평가하는데, 이 역시 문제 풀이 후 프레젠테이션 및 질의응답 순으로 진행한다. 일반적인 방법 외에 자신만의 창의적인 방법을 제시하되, 무조건 창의적인 생각으로 답변해야 한다는 강박관념을 가지기보다는 논리적이고 일관성 있는 대답을 준비하도록 한다.#면접 후기_더 보기#최종 합격 후기_더 보기
]

질문
[
회사의 인재상을 알려줘
]
"""
formatted_prompt = f"instruction: {prompt}\n output:"   

In [91]:
formatted_prompt

'instruction: 마지막 질문에 답변하기 위해 다음 context를 사용하세요.\n답을 모르겠으면 모르겠다고 말하세요. 답변을 지어낼려고 노력하지 마세요.\n\ncontext\n[\n삼성전자는 부문별로 나눠 신입 사원 및 경력 사원을 채용하고 있다. 신입 사원 모집은 상반기에는 3월, 하반기에는 9월에 지원서 접수가 시작된다. 특정 분야 인력이 필요할 경우, 삼성전자 홈페이지 및 주요 취업 사이트를 통해 채용 공고를 게재하고 있다. 또한 수시 모집도 진행하고 있으니 경력 지원서는 언제든지 등록하면 된다. 모든 채용은 삼성 채용 홈페이지를 통해 접수가 가능하다. 3급 신입 공채 지원은 졸업 예정자 또는 기 졸업자를 대상으로 한다. 병역필 또는 면제자로 해외여행에 결격 사유가 없어야 하고, 어학 자격(OPIc 또는 TOEIC Speaking)을 보유하고 있어야 한다.삼성전자, 기업 개요삼성전자는 본사를 거점으로 한국과 DX 부문 산하 해외 9개 지역총괄, DS 부문 산하 해외 5개 지역총괄의 생산/  및 SDC, Harman 산하 종속기업 등 232개의 종속기업으로 구성된 글로벌 전자 기업이다. 2021년 12월 조직 개편을 통해 CE 부문과 IM 부문을 DX 부문으로 통합했으며, 내부 조직 체계에 맞춰 SDC를 DS 부문과 별도로 구분했다. 제품의 특성에 따라 DX(Device eXperience), DS(Device Solutions) 2개의 부문과 패널 사업을 영위하는 SDC(삼성디스플레이㈜ 및 그 종속기업), 전장부품 사업 등을 영위하는 Harman(Harman International Industries, Inc. 및 그 종속기업)으로 나눠 경영을 하고 있다.#기업 정보_더보기삼성전자, 인재상열정(Passion)끊임없는 열정으로 미래에 도전하는 인재창의혁신(Creativity)창의와 혁신으로 세상을 변화시키는 인재인간미/도덕성(Integrity)정직과 바른 행동으로 역할과 책임을 다하는 인재1. 서류 전형 패스하기지원서에 기본 인적 사항, 학업 이

In [90]:
llm.generate(formatted_prompt)

ValueError: Argument 'prompts' is expected to be of type List[str], received argument of type <class 'str'>.

In [ ]:
# 변수 설정
company_name = "삼성전자"

# CSV 파일 경로
csv_file_path = "./company_talents.csv"

# CSV 파일 읽기
df = pd.read_csv(csv_file_path)

# # 파이프라인 객체 생성
# hf_pipeline = pipeline("text-generation", model=model_orion, tokenizer=tokenizer_orion, streamer=streamer)
    
# # HuggingFacePipeline으로 모델 래핑
# llm = HuggingFacePipeline(pipeline=hf_pipeline)

# company_name 열에 company_name이 있는지 확인
if company_name in df["company_name"].values:
    # company_name이 존재하는 경우, talents 열의 값을 가져옴
    talents = df[df["company_name"] == company_name]["talents"].tolist()
    # 결과 출력
    print("Talents:", talents)
else:
    # company_name이 존재하지 않는 경우, extract_company_info 함수 실행
    company_info = extract_company_info(company_name)
    print('컴인포', company_info[1]);
    if company_info == 'fail':
        print("회사 못찾음")
    else :
                # 문서 임베딩
        vector_store = FAISS.from_texts(texts=[company_info[0]], embedding=embeddings)

        # Retriever 생성
        retriever = vector_store.as_retriever(search_kwargs={"k": 1})

        # 인재상 생성
        talents = generate_talent_description(company_name, retriever, llm)
        print(talents)
        print(type(talents))
        str(talents)

        # 결과를 데이터프레임에 추가
        new_data = {"company_name": company_name, "talents": talents}
        df = pd.concat([df, pd.DataFrame([new_data])], ignore_index=True)
        
        # 결과를 CSV 파일에 저장
        # df.to_csv(csv_file_path, index=False)
        print("인재상이 생성되어 CSV 파일에 추가되었습니다.")